In [ ]:
asset_name = input("Enter asset name (default: Gold in Mumbai): ") or "Gold in Mumbai"
query = f"price of {asset_name}"
print(f"Searching for: {query}")



In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

def get_google_snippets(query, num_results=5):
    # Setup headless browser
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--lang=en-US")  # Ensure English results
    driver = webdriver.Chrome(options=options)

    try:
        # Load the Google search results page
        driver.get(f"https://www.google.com/search?q={query.replace(' ', '+')}")
        time.sleep(10)  # Let page load fully

        snippets = []
        # Get organic search results
        result_blocks = driver.find_elements(By.CSS_SELECTOR, 'div.g')

        for result in result_blocks:
            try:
                title_element = result.find_element(By.TAG_NAME, 'h3')
                link_element = result.find_element(By.TAG_NAME, 'a')
                snippet_element = result.find_element(By.CSS_SELECTOR, 'div.VwiC3b')  # snippet text class

                title = title_element.text
                url = link_element.get_attribute('href')
                snippet = snippet_element.text

                snippets.append({
                    'title': title,
                    'url': url,
                    'snippet': snippet
                })

                if len(snippets) >= num_results:
                    break

            except Exception:
                continue

        return snippets

    finally:
        driver.quit()

# Example
result= get_google_snippets(query)


In [20]:
for title, url, snippet in result:
    print(f"{title}\n{url}\n{snippet}\n{'='*40}")


In [21]:
len(result)


0

In [ ]:
# --  Extracting Additional data from URLs using BeautifulSoup and requests
from googlesearch import search

def get_top_links(query, num=5):
    return list(search(query, num_results=num))

top_links = get_top_links(query)


In [9]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(response.text, "html.parser")
        return soup.get_text(separator="\n")
    except Exception as e:
        return ""
extracted_texts = [extract_text_from_url(link) for link in top_links]

In [10]:
print(f"Top {len(extracted_texts)} links for '{query}':")
print("\n".join(top_links))
for i, text in enumerate(extracted_texts, 1):
    print(f"\nLink {i}:\n{text[:500]}...")  # Print first 500 characters for brevity

Top 5 links for 'price of water in germany today':
https://www.reddit.com/r/germany/comments/u0qmha/whats_the_deal_with_the_price_of_water_at_german/
https://www.waternewseurope.com/water-prices-compared-in-36-eu-cities/
https://www.numbeo.com/cost-of-living/country_price_rankings?itemId=7
https://www.selinawamucii.com/insights/prices/germany/mineral-water/
https://worldpopulationreview.com/country-rankings/bottled-water-cost-by-country

Link 1:








Reddit - The heart of the internet




























































 
















Skip to main content






















































Open menu




Open navigation








































































































































 


Go to Reddit Home










































r/germany




A chip


















A close button




















...

Link 2:
 
 
 
 
Water prices compared in 36 E

In [ ]:
import ollama

def summarize_content_ollama(asset, text, model="mistral"):
    prompt = f"""
You are an asset valuation expert.

Given this text, estimate the current market value of "{asset}" and summarize the reasoning briefly.

TEXT:
{text[:4000]}

Respond only in JSON format like this:
{{
  "asset": "{asset}",
  "value": <number>,
  "note": "<brief summary of key points>"
}}
"""
    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    return response['message']['content']




In [ ]:
# save json output and print it
import json
report=summarize_content_ollama(asset_name, extracted_texts[0], model="mistral")
print("Final Report:"+ json.dumps(json.loads(report), indent=2))